# Naive Bayes : Cas de détection de spam

In [1]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('emails/spam', 'spam'),sort='false')
data = data.append(dataFrameFromDirectory('emails/ham', 'ham'),sort='false')


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17804\2605442815.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(dataFrameFromDirectory('emails/spam', 'spam'),sort='false')
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17804\2605442815.py:37: FutureWarning: Passing non boolean values for sort is deprecated and will error in a future version!
  data = data.append(dataFrameFromDirectory('emails/spam', 'spam'),sort='false')
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17804\2605442815.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(dataFrameFromDirectory('emails/ham', 'ham'),sort='false')
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17804\2605442815.py:38: FutureWarning: Passing non boolean values for sort is deprecated and will error in a future version!
  data = data.a

In [2]:
data.head(10)

,class,message
emails/spam\00001.7848dde101aa985090474a91ec93fcf0,spam,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
emails/spam\00004.eac8de8d759b7e74154f142194282724,spam,##############################################...
emails/spam\00005.57696a39d7d84318ce497886896bf90d,spam,I thought you might like these:\n\n1) Slim Dow...
emails/spam\00006.5ab5620d3d7c6c0db76234556a16f6c1,spam,A POWERHOUSE GIFTING PROGRAM You Don't Want To...
emails/spam\00007.d8521faf753ff9ee989122f6816f87d7,spam,Help wanted. We are a 14 year old fortune 500...
emails/spam\00008.dfd941deb10f5eed78b1594b131c9266,spam,<html>\n\n<head>\n\n<title>ReliaQuote - Save U...
emails/spam\00009.027bf6e0b0c4ab34db3ce0ea4bf2edab,spam,TIRED OF THE BULL OUT THERE?\n\nWant To Stop L...
emails/spam\00010.445affef4c70feec58f9198cfbc22997,spam,"Dear ricardo1 ,\n\n\n\n<html>\n\n<body>\n\n<ce..."


In [3]:
Y=data['class']
X=data['message']
print(data['message'][0])

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">

<HTML><HEAD>

<META content=3D"text/html; charset=3Dwindows-1252" http-equiv=3DContent-T=

ype>

<META content=3D"MSHTML 5.00.2314.1000" name=3DGENERATOR></HEAD>

<BODY><!-- Inserted by Calypso -->

<TABLE border=3D0 cellPadding=3D0 cellSpacing=3D2 id=3D_CalyPrintHeader_ r=

ules=3Dnone 

style=3D"COLOR: black; DISPLAY: none" width=3D"100%">

  <TBODY>

  <TR>

    <TD colSpan=3D3>

      <HR color=3Dblack noShade SIZE=3D1>

    </TD></TR></TD></TR>

  <TR>

    <TD colSpan=3D3>

      <HR color=3Dblack noShade SIZE=3D1>

    </TD></TR></TBODY></TABLE><!-- End Calypso --><!-- Inserted by Calypso=

 --><FONT 

color=3D#000000 face=3DVERDANA,ARIAL,HELVETICA size=3D-2><BR></FONT></TD><=

/TR></TABLE><!-- End Calypso --><FONT color=3D#ff0000 

face=3D"Copperplate Gothic Bold" size=3D5 PTSIZE=3D"10">

<CENTER>Save up to 70% on Life Insurance.</CENTER></FONT><FONT color=3D#ff=

0000 

face=3D"Copperplate Gothic Bold" size=3D5 PT

In [4]:
print(data['message'][0].split())

['<!DOCTYPE', 'HTML', 'PUBLIC', '"-//W3C//DTD', 'HTML', '4.0', 'Transitional//EN">', '<HTML><HEAD>', '<META', 'content=3D"text/html;', 'charset=3Dwindows-1252"', 'http-equiv=3DContent-T=', 'ype>', '<META', 'content=3D"MSHTML', '5.00.2314.1000"', 'name=3DGENERATOR></HEAD>', '<BODY><!--', 'Inserted', 'by', 'Calypso', '-->', '<TABLE', 'border=3D0', 'cellPadding=3D0', 'cellSpacing=3D2', 'id=3D_CalyPrintHeader_', 'r=', 'ules=3Dnone', 'style=3D"COLOR:', 'black;', 'DISPLAY:', 'none"', 'width=3D"100%">', '<TBODY>', '<TR>', '<TD', 'colSpan=3D3>', '<HR', 'color=3Dblack', 'noShade', 'SIZE=3D1>', '</TD></TR></TD></TR>', '<TR>', '<TD', 'colSpan=3D3>', '<HR', 'color=3Dblack', 'noShade', 'SIZE=3D1>', '</TD></TR></TBODY></TABLE><!--', 'End', 'Calypso', '--><!--', 'Inserted', 'by', 'Calypso=', '--><FONT', 'color=3D#000000', 'face=3DVERDANA,ARIAL,HELVETICA', 'size=3D-2><BR></FONT></TD><=', '/TR></TABLE><!--', 'End', 'Calypso', '--><FONT', 'color=3D#ff0000', 'face=3D"Copperplate', 'Gothic', 'Bold"', 'siz

In [5]:
from sklearn.model_selection import train_test_split

vectorizer = CountVectorizer()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=1)
counts = vectorizer.fit_transform(X_train.values)
vectorizer.vocabulary_
#print(counts.toarray())

{'this': 44313,
 'is': 26133,
 'multi': 31656,
 'part': 34642,
 'message': 30606,
 'in': 25255,
 'mime': 30873,
 'format': 20805,
 '_nextpart_3402248108': 6685,
 'content': 14469,
 'type': 45416,
 'text': 44123,
 'plain': 35737,
 'charset': 13088,
 'iso': 26175,
 '8859': 5746,
 'transfer': 44962,
 'encoding': 18655,
 'quoted': 37772,
 'printable': 36504,
 '93connecting': 6128,
 'your': 50171,
 'business': 11983,
 'to': 44645,
 'the': 44208,
 'world': 48876,
 'wide': 48520,
 'web': 48230,
 '94': 6131,
 'how': 23899,
 'many': 30072,
 'shoppers': 41248,
 'are': 9125,
 'you': 50162,
 'losing': 29357,
 'figure': 20283,
 'would': 48911,
 'amaze': 8485,
 'them': 44230,
 'they': 44275,
 'cannot': 12489,
 'find': 20335,
 'site': 41515,
 'simple': 41452,
 'equation': 18966,
 'not': 32796,
 'being': 10570,
 'found': 20870,
 '3d': 2998,
 'new': 32368,
 'customers': 15194,
 'we': 48209,
 'can': 12461,
 'change': 13035,
 'that': 44201,
 'for': 20733,
 'only': 33681,
 '119': 881,
 '97': 6185,
 'will'

In [7]:
classifier = MultinomialNB()
#targets = data['class'].values
targets = y_train.values
#print(targets)
classifier.fit(counts, targets)

MultinomialNB()

In [8]:
from sklearn.metrics import accuracy_score

counts_test = vectorizer.transform(X_test.values)
predictions = classifier.predict(counts_test)
print('Naivebayes Classifier (accuracy): '+str(accuracy_score(predictions,y_test)))

Naivebayes Classifier (accuracy): 0.9733333333333334


In [9]:
examples = ["Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
prediction = classifier.predict(example_counts)
prediction

array(['ham'], dtype='<U4')

In [10]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predictions))

[[501   2]
 [ 14  83]]
